<a href="https://colab.research.google.com/github/igvilla/Predict_FingerFlexionAngle_from_ECoG/blob/main/Updated_Leaderboard_Submission_Final_Competition_Stuff_ElasticNet%2BRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BE 521: Final Project Part 1

Collaborators (Group Members): Anusha Keshireddy, Jennifer Luo, Ignacio Villasmil 

Spring 2022

Adapted by Kevin Xie

32 Points

Objective: Predict finger movements from ECoG Recordings

Due: March 31st


# 1. Getting Started (4 pts)
The following sections will walk you through the development of the prediction pipeline. 

In [ ]:
#Set up the notebook environment
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from scipy.stats import pearsonr
from scipy import signal as sig

import pdb #for breakpoints

## 1.
Extract the dataglove and ECoG data for each subject from the pickle file. Feel free to copy the code snippet above. Split the data into a training and testing set (at least 50% of the data should be in the training set). 

**How many samples are there in the full ECoG recording (before splitting)?** (1 pt)

**How many samples do you have in your training set? In your testing set?** (1 pt)

In [ ]:
#for Jennifer
from google.colab import drive
drive.mount('/content/drive') #get drive

#change depending on person? 
%cd /content/drive/MyDrive/Spring 2022/BE521/Final

Mounted at /content/drive
/content/drive/MyDrive/Spring 2022/BE521/Final


In [ ]:
# for Iggy:

from google.colab import drive
drive.mount('/content/drive') #get drive

#change depending on person? 
%cd /content/drive/MyDrive/BE521/Final

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1z5dynlKdNP0PkOgS8xVONN9OwfUOLKIU/Final


In [ ]:
# for Anusha:

from google.colab import drive
drive.mount('/content/drive') #get drive

#change depending on person? 
%cd /content/drive/MyDrive/Final

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1z5dynlKdNP0PkOgS8xVONN9OwfUOLKIU/Final


In [ ]:
import scipy.io as sio
proj_data=sio.loadmat('raw_training_data.mat')

In [ ]:
#splitting into testing and training sets

#patient 1

#glove
total_p1_glove=proj_data['train_dg'][0][0]
training_p1_glove=total_p1_glove[0:int(300000*0.6),:] #60% of samples in training
testing_p1_glove=total_p1_glove[int(300000*0.6):300000,:] #remaining 40% in testing
#ecog
total_p1_ecog=proj_data['train_ecog'][0][0]
training_p1_ecog=total_p1_ecog[0:int(300000*0.6),np.arange(62)!=54] #60% of samples in training
testing_p1_ecog=total_p1_ecog[int(300000*0.6):300000,np.arange(62)!=54] #remaining 40% in testing

#patient 2

#glove
total_p2_glove=proj_data['train_dg'][1][0]
training_p2_glove=total_p2_glove[0:int(300000*0.6),:] #60% of samples in training
testing_p2_glove=total_p2_glove[int(300000*0.6):300000,:] #remaining 40% in testing
#ecog
total_p2_ecog=proj_data['train_ecog'][1][0]
training_p2_ecog=total_p2_ecog[0:int(300000*0.6),np.logical_and(np.arange(48)!=20,np.arange(48)!=37)] #60% of samples in training
testing_p2_ecog=total_p2_ecog[int(300000*0.6):300000,np.logical_and(np.arange(48)!=20,np.arange(48)!=37)] #remaining 40% in testing

#patient 3

#glove
total_p3_glove=proj_data['train_dg'][2][0]
training_p3_glove=total_p3_glove[0:int(300000*0.6),:] #60% of samples in training
testing_p3_glove=total_p3_glove[int(300000*0.6):300000,:] #remaining 40% in testing
#ecog
total_p3_ecog=proj_data['train_ecog'][2][0]
training_p3_ecog=total_p3_ecog[0:int(300000*0.6),:] #60% of samples in training
testing_p3_ecog=total_p3_ecog[int(300000*0.6):300000,:] #remaining 40% in testing


There are 300,000 samples in the full ECOG recording per patient before splitting into testing and training sets. 60% of the samples were put into training set, and 40% in testing set. This corresponded to 180,000 training samples per patient, and 120,000 testing samples per patient.

## 2.
Next, complete the `filter_data` function. Test it using the raw data extracted in the prior step. What filter types and cutoff frequencies did you use? (2 pts)

In [ ]:
def filter_data(raw_eeg, srate=1000, cutoffs=[75, 200]):
  """
  Write a filter function to clean underlying data.
  Filter type and parameters are up to you. Points will be awarded for reasonable filter type, parameters and application.
  Please note there are many acceptable answers, but make sure you aren't throwing out crucial data or adversly
  distorting the underlying data!

  Input: 
    raw_eeg (samples x channels): the raw signal
    fs: the sampling rate (1000 for this dataset)
  Output: 
    clean_data (samples x channels): the filtered signal
  """

  numChan = np.shape(raw_eeg)[1] #number of channels
  filteredData = np.zeros(np.shape(raw_eeg)); #filtered data output

  #butterworth filter of 4th order
  sos = sig.butter(4, cutoffs, 'bandpass', analog=False, fs=srate, output='sos'); # returns filter coefficients

  #for each channel
  for chanInd in np.arange(numChan):
    #mean subtract data
    currFilt = raw_eeg[:, chanInd] - np.mean(raw_eeg[:, chanInd]);
    #filter between 075 and 115 hz with a filtfilt (no phase distortion)
    currFilt = sig.sosfiltfilt(sos, currFilt) # forward-backward digital filter using cascaded second-order sections
                                              # sos = filter coefficients ... currFilt = data to be filtered
    filteredData[:, chanInd] = currFilt
  return filteredData

According to Kubanek 2009, frequencies above 50 Hz are relevant to finger movement decoding. When a fast-fourier transform of the data was computed, there was a high power at 60 Hz, which likely corresponds to electrical noise. Thus, 50-75 Hz was avoided. A 4th order butterworth bandpass filter with cutoff frequencies 75 and 200Hz was used. The filter was run fowards and backwards over the signal with filtfilt to remove phase distortion. 

The cutoff frequencies of 75-115 Hz were chosen to isolate the high gamma band, which showed significant activity changes according to Kubanek 2009. Therefore, while an general cutoff was initialized to be 75-200 Hz, an additional argument was added in the case that other frequency bands had useful features, including the gamma band. 

It is also important to note that there is a resonant frequency of 60 Hz noise at 120 Hz, so this frequency should be avoided when extracting features in relevant frequency bands. 

**The following section is us exploring the data. **

# 2. Calculating Features (12 points)

Here you will complete the `get_windowed_feats` and `get_features` functions. 

## 1. 
We will calculate features across sliding time windows. if we use a suggested window length of 100ms with a 50ms window overlap, how many feature windows, $M$, will we have if we computed features using all the data in a given subject? Feel free to re-use code from previous homeworks. 

In [ ]:
#this function returns the number of windows and the remainder time (outside window)

def NumWins(x,fs,winLen,winDisp):
  #all times in seconds

  #calculate number of samples
  xLen=len(x)

  #calculate length of signal in time (seconds)
  xtime=xLen/fs

  #calculate winDisp and winLen in samples
  winLen_samples=round(winLen*fs) #window length in samples
  winDisp_samples=round(winDisp*fs) #window displacement in samples

  #calculate number of windows and the remainder of samples left over. 

  #if not overlapping
  #if winDisp==winLen:
    #NumWins=xtime//winDisp #when not overlapping, the length of the window=window displacement. Divide by total signal by this. 
    #rem_windows=xtime-(NumWins*winDisp) #answer given in time

  #if there is overlap 
  #if winLen>winDisp:
    #NumWins=(xtime-(winLen-winDisp))//winDisp #when overlapping, remainder+(windowlength-displacement)+displacement*num_windows=length of signal
    #rem_windows=xtime-(winLen-winDisp)-(NumWins*winDisp) #answer given in time
  
  #if not overlapping
  if winDisp_samples==winLen_samples:
    NumWins=xLen//winDisp_samples #when not overlapping, the length of the window=window displacement. Divide by total signal by this. 
    rem_windows=xLen-(NumWins*winDisp_samples) #answer given in samples

  #if there is overlap 
  if winLen_samples>winDisp_samples:
    NumWins=(xLen-(winLen_samples-winDisp_samples))//winDisp_samples #when overlapping, remainder+(windowlength-displacement)+displacement*num_windows=length of signal
    rem_windows=xLen-(winLen_samples-winDisp_samples)-(NumWins*winDisp_samples) #answer given in samples

  return [NumWins,rem_windows]

There are 5998 feature windows.

## 2.
Now complete the `get_features` function. Please create **4 or more** different features to calculate for each channel in each time window. Features may include the average time-domain voltage, or the average frequency-domain magnitude in consecutive 15Hz frequency bands, bandpower of relevant frequency bands, etc...

In [ ]:
def time_avg(signal):
  return np.mean(signal)

In [ ]:
from numpy.fft import fft, ifft

def band_power(signal,fs,low,high):
  X = fft(signal)
  N = len(X)
  n = np.arange(N)
  T = N/fs #sampling rate=1000
  freq = n/T 
  power_spectrum=np.abs(X)

  #to get average bandpower, find frequencies between a certain range and integrate between them

  # Find values in frequency vector corresponding to gamma band
  index_band = np.logical_and(freq >= low, freq <= high)
  from scipy.integrate import simps

  # Frequency resolution
  #freq_res = freq[1] - freq[0]  #diff between frequency values

  # Compute the absolute power by approximating the area under the curve
  #band_power = simps(power_spectrum[index_band], dx=freq_res)

  #average frequency domain magnitude
  avg_freq_domain_mag=np.mean(power_spectrum[index_band])

  return avg_freq_domain_mag

Possible new features to try: -Take the filtered window and filter again- specifically with the gamma band (75-115 Hz). -----calculate time domain magnitude in this band -----calculate frequency domain magnitude in this band

-repeat for 125-160 Hz, 160-200 Hz.

These features may be useful instead of energy, line length, and zero crossings, which may not be very useful.

In [ ]:
def get_features(filtered_window, fs=1000):
  """
    Write a function that calculates features for a given filtered window. 
    Feel free to use features you have seen before in this class, features that
    have been used in the literature, or design your own!

    Input: 
      filtered_window (window_samples x channels): the window of the filtered ecog signal 
      fs: sampling rate
    Output:
      features (channels x num_features): the features calculated on each channel for the window
  """

  [window_samples,num_channels]=np.shape(filtered_window)

  #initialize features array, shape=(num_channels,num_features). 
  #tells us the features for a filtered window (all channels of that window)

  #for now, I'm assuming I have six features, I will add more later

  features=np.empty((num_channels,7))

  for cc in range(num_channels):

    #get the window for each channel
    curr_window=filtered_window[:,cc] 

    #calculate the features for that channel, window and add to matrix
    #add additional features later!!!!

    freq_dom_mag_8_12=band_power(curr_window,1000,8,12)
    #freq_dom_mag_20_25=band_power(curr_window,1000,20,25)
    low_gamma_freq_dom_mag=band_power(curr_window,1000,75,95)
    high_gamma_freq_dom_mag=band_power(curr_window,1000,96,115)
    freq_dom_mag_146_160=band_power(curr_window,1000,146,160)
    freq_dom_mag_125_145=band_power(curr_window,1000,125,145)
    freq_dom_mag_160_200=band_power(curr_window,1000,160,175)


    features[cc,0]=time_avg(curr_window)
    features[cc,6]=freq_dom_mag_8_12
    features[cc,1]=low_gamma_freq_dom_mag
    features[cc,2]=freq_dom_mag_125_145
    features[cc,3]=freq_dom_mag_160_200
    features[cc,4]=high_gamma_freq_dom_mag
    features[cc,5]=freq_dom_mag_146_160
    

  
  return features


## 3.
Now finish the `get_windowed_feats` function by putting the `filter_data` and `get_features` functions together to return a feature vector for each time window.

In [ ]:
#all times in seconds
def get_windowed_feats(raw_ecog, fs, window_length, window_overlap):

  clean_data=filter_data(raw_ecog, fs) #outputs filtered data (samples x channels)
  [num_samples,num_channels]=np.shape(clean_data)

  #split the filtered data into windows- for every single channel. For each window, calculate features. 
  
  #for numWins function- need to input only one channel of data
  x=clean_data[:,0] #takes the first channel

  [numWindows,rem_samples]=NumWins(x,fs,window_length,window_overlap) #calculate number of windows and remaining samples

  #convert everything to units of samples
  winLen_samples=round(window_length*fs) #window length in samples
  winDisp_samples=round(window_overlap*fs) #window displacement in samples 

  #initialize empty feature vector- length of number of windows
  all_feats=np.zeros([round(numWindows),num_channels*7]) 

  #starting index of x- first window occurs after remainder samples since right aligned
  window_start=rem_samples

  for ww in range(round(numWindows)): #for each window
    window_end=window_start+winLen_samples
    filtered_window=clean_data[window_start:window_end,:] #signal in that window, all channels
    
    #find features of that window for all channels
    features=get_features(filtered_window, fs=1000) 

    #add collapsed feature array into the allfeats array
    #we will do feature 1, all channels, then feature 2, all channels, etc.
    all_feats[ww,:]=features.flatten('F') #flattens by column so get the structure we want

    #set new starting index of window
    window_start=window_start+winDisp_samples #prior start + window displacement
  
  return all_feats

  """
    Write a function which processes data through the steps of filtering and
    feature calculation and returns features. Points will be awarded for completing
    each step appropriately (note that if one of the functions you call within this script
    returns a bad output, you won't be double penalized). Note that you will need
    to run the filter_data and get_features functions within this function. 

    Inputs:
      raw_eeg (samples x channels): the raw signal
      fs: the sampling rate (1000 for this dataset)
      window_length: the window's length
      window_overlap: the window's overlap
    Output: 
      all_feats (num_windows x (channels x features)): the features for each channel for each time window
        note that this is a 2D array. 
  """

# 3. Creating the Response Matrix (6 points)
In this section, you will develop code for your *create_R_matrix* function

## 2. 
We do not have feature data to fill out the first N-1 data rows in the R matrix that will be used to predict the first N-1 finger angles. One way to work around this is to append a copy of the first N-1 rows of your feature matrix to the beginning of your feature matrix before calculating R. Make this adjustment in `create_R_matrix`, then compute the response matrix R. You can test whether your function is running correctly by running `create_R_matrix` with data from `testRfunction.pkl` using 3 windows and verifying that the quantity `np.mean(R)` is 25.4668 (5 points).

In [ ]:
def create_R_matrix(features, N_wind):
  """ 
  Write a function to calculate the R matrix

  Input:
    features (samples (number of windows in the signal) x channels x features): 
      the features you calculated using get_windowed_feats
    N_wind: number of windows to use in the R matrix

  Output:
    R (samples x (N_wind*channels*features))
  """
  # first, add N-1 rows to the features matrix:
  
  # featMat[0] = first row
  # featMat[1] = second row

  new_firstRow = features[0]
  new_secondRow = features[1]

  features_appended = np.vstack([new_secondRow, features])  # adding second row to the top of the matrix first --> assigning it to a new variable
  features_appended = np.vstack([new_firstRow, features_appended])  # adding first row to the top of the NEW matrix now (ontop of the added second row)

  # INITIALIZE EMPTY R MATRIX:

  # N_wind = number of samples / time bins for the R matrix = 3
      # provided as an input

  # samples (number of windows in the signal) = number of rows 
  samples = len(features)   # number of rows = number of windows

  # ch = number of channels

  # USED TO BE np.empty...
  R = np.zeros((samples, (N_wind*len(features[0,:]))))  # len(features[0,:]) = (num of features)*(num of channels)
                                                        # N_wind * len(features[0,:]) = (num of samples or time bins)*(num of features)*(num of channels)

  f = 0
  s = 1
  t = 2

  for i in range(len(features[0,:])):   # goes thru each column of the features matrix
    # append every time bin (aka sample) for every column of the features matrix

    R[:, f] = features_appended[0 : (len(features_appended)-2), i]
    R[:, s] = features_appended[1 : (len(features_appended)-1), i]
    R[:, t] = features_appended[2 : len(features_appended), i]

    f += 3
    s += 3
    t += 3

    #############

    # R = np.append(R, features[0 : (len(features_appended)-2), i])  
    # R = np.append(R, features[1 : (len(features_appended)-1), i])
    # R = np.append(R, features[2 : len(features_appended), i]) 

  return R

# 4. ML Training and Testing (10 points)
Here we will use the optimal linear decoder framework to predict finger angles, and additionally you will use one or more classifiers of your own choosing to make the prediction.

In [ ]:
#FOR ALL PATIENTS- NEED TO STANDARDIZE THE FEATURES

def feat_standardize(feature_matrix,test_feature_matrix):

  [windows,feats]=np.shape(feature_matrix)
  [testwindows,feats_test]=np.shape(test_feature_matrix)
  normFeats=np.empty((windows,feats))
  testnormFeats=np.empty((testwindows,feats_test))

  for nn in range(feats):
    mean_train=np.mean(feature_matrix[:,nn])
    std_train=np.std(feature_matrix[:,nn])
    normFeats[:,nn]=(feature_matrix[:,nn]-mean_train)/std_train
    mean_test=np.mean(test_feature_matrix[:,nn])
    std_test=np.std(test_feature_matrix[:,nn])
    testnormFeats[:,nn]=(test_feature_matrix[:,nn]-mean_train)/std_train #normalize test matrix by same mean/std as training 

  
  return([normFeats,testnormFeats])



In [ ]:
import sklearn as sk
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
def zoInterp(x, numInterp):
  #return np.reshape(np.tile(x,numInterp),numInterp,'F')

  repeat=np.tile(x[0],numInterp)
  interpolation_x=repeat
  for p in range(1,len(x)):
    repeat=np.tile(x[p],numInterp)
    interpolation_x=np.hstack((interpolation_x,repeat))
  return interpolation_x

In [ ]:
window_length=100e-3
window_overlap=50e-3
fs=1000

#window overlap in samples is amount to repeat each prediction 
winDisp_samples=round(window_overlap*fs) #window displacement in samples 
winLen_samples=round(window_length*fs) #window length in samples

[M,rem_samples]=NumWins(testing_p1_glove[:,0],fs,window_length,window_overlap) #calculate number of windows and remaining time

#remaining indices- need to pad
rem_indices=rem_samples+winLen_samples-winDisp_samples


In [ ]:
def smooth(input, smoothSize):
  #convThing = np.ones(smoothSize)/smoothSize;
  convThing = sig.gaussian(smoothSize, 3)/smoothSize;
  return np.convolve(input, convThing, mode='same')

# Leaderboard Submission

In [ ]:
#Import leaderboard ecog data

import scipy.io as sio
leaderboard_data=sio.loadmat('leaderboard_data.mat')


In [ ]:
leaderboard_data['leaderboard_ecog'][0]
p1_board_ecog=leaderboard_data['leaderboard_ecog'][0][0]
p2_board_ecog=leaderboard_data['leaderboard_ecog'][1][0]
p3_board_ecog=leaderboard_data['leaderboard_ecog'][2][0]

In [ ]:
#check that the same channels are bad
#plot all channels

In [ ]:
#pre-processing: removing bad channels

p1_ecog=p1_board_ecog[:,np.arange(62)!=54] #remove channel 55
p2_ecog=p2_board_ecog[:,np.logical_and(np.arange(48)!=20,np.arange(48)!=37)] #remove channel 21 and 38
p3_ecog=p3_board_ecog


In [ ]:
#filter, calculate features on the ecog

window_length=100e-3
window_overlap=50e-3
fs=1000

#calculate features for training data, call it p1_trainFeats
p1_Feats_l=get_windowed_feats(p1_ecog, fs, window_length, window_overlap)
p2_Feats_l=get_windowed_feats(p2_ecog, fs, window_length, window_overlap)
p3_Feats_l=get_windowed_feats(p3_ecog, fs, window_length, window_overlap)

In [ ]:
#get entire raw ecog training data and glove data to train the model


#Patient 1
#glove
total_p1_glove=proj_data['train_dg'][0][0]
#ecog
total_p1_ecog=proj_data['train_ecog'][0][0]
raw_train_p1_ecog=total_p1_ecog[:,np.arange(62)!=54] 

#patient 2

#glove
total_p2_glove=proj_data['train_dg'][1][0]
#ecog
total_p2_ecog=proj_data['train_ecog'][1][0]
raw_train_p2_ecog=total_p2_ecog[:,np.logical_and(np.arange(48)!=20,np.arange(48)!=37)] 

#patient 3

#glove
total_p3_glove=proj_data['train_dg'][2][0]
#ecog
total_p3_ecog=proj_data['train_ecog'][2][0]
raw_train_p3_ecog=total_p3_ecog


In [ ]:
#calculate new Y_train for each patient w the entire training data

#calculate Y for the training data- basically flexion data downsampled to be Y=Mx5. ALL PATIENTS

[M,rem_samples]=NumWins(total_p1_glove[:,0],fs,window_length,window_overlap) #calculate number of windows and remaining time

#convert everything to units of samples
winLen_samples=round(window_length*fs) #window length in samples
winDisp_samples=round(window_overlap*fs) #window displacement in samples 

#starting index of x- first window occurs after remainder samples since right aligned
window_start=rem_samples

#initialize- patient 1
flex_data=np.empty([int(M),5])

for mm in range(round(M)): #for each window
  flex_data[mm,:]=total_p1_glove[window_start,:] #find the flexion data for each finger at that window from glove data for P1
  window_start=window_start+winDisp_samples #move to next window

Y_train_1_all=flex_data
print(np.shape(Y_train_1_all))

#initialize- patient 2
flex_data=np.empty([int(M),5])
window_start=rem_samples
for mm in range(round(M)): #for each window
  flex_data[mm,:]=total_p2_glove[window_start,:] #find the flexion data for each finger at that window from glove data for P1
  window_start=window_start+winDisp_samples #move to next window

Y_train_2_all=flex_data
print(np.shape(Y_train_2_all))

#initialize- patient 3
flex_data=np.empty([int(M),5])
window_start=rem_samples
for mm in range(round(M)): #for each window
  flex_data[mm,:]=total_p3_glove[window_start,:] #find the flexion data for each finger at that window from glove data for P1
  window_start=window_start+winDisp_samples #move to next window

Y_train_3_all=flex_data
print(np.shape(Y_train_3_all))

(5999, 5)
(5999, 5)
(5999, 5)


In [ ]:
#calculate features for the raw training data we will use to train model

window_length=100e-3
window_overlap=50e-3
fs=1000

#calculate features for training data, call it p1_trainFeats
p1_alltrainFeats=get_windowed_feats(raw_train_p1_ecog, fs, window_length, window_overlap)
p2_alltrainFeats=get_windowed_feats(raw_train_p2_ecog, fs, window_length, window_overlap)
p3_alltrainFeats=get_windowed_feats(raw_train_p3_ecog, fs, window_length, window_overlap)


In [ ]:
#training and testing features normalize for all patients
#use entire training data given
[p1_FeatNorm_train,p1_FeatNorm_l]=feat_standardize(p1_alltrainFeats,p1_Feats_l)
[p2_FeatNorm_train,p2_FeatNorm_l]=feat_standardize(p2_alltrainFeats,p2_Feats_l)
[p3_FeatNorm_train,p3_FeatNorm_l]=feat_standardize(p3_alltrainFeats,p3_Feats_l)

In [ ]:
#Calculate R matrix w 3 windows for each of the 3 patients normalized features for leaderboard
R_P1_l=create_R_matrix(p1_FeatNorm_l,3) #N=3
R_P2_l=create_R_matrix(p2_FeatNorm_l,3) #N=3
R_P3_l=create_R_matrix(p3_FeatNorm_l,3) #N=3



In [ ]:
#calculate R matrix w 3 windows for each of 3 patients normalized features----- all training data

R_P1_train=create_R_matrix(p1_FeatNorm_train,3) #N=3
R_P2_train=create_R_matrix(p2_FeatNorm_train,3) #N=3
R_P3_train=create_R_matrix(p3_FeatNorm_train,3) #N=3


In [ ]:
def smooth(input, smoothSize):
  #convThing = np.ones(smoothSize)/smoothSize;
  convThing = sig.gaussian(smoothSize, 3)/smoothSize;
  return np.convolve(input, convThing, mode='same')

In [ ]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
import sklearn as sk

from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import model_selection
from sklearn.linear_model import RidgeCV, LassoCV,ElasticNetCV

# elastic net:
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression

In [ ]:
#models and output predictions- elastic net

samples,channels=np.shape(p1_board_ecog)

#initialize arrays for glove prediction (num samples by 5 fingers)
Patient_3_Prediction=np.zeros((samples,5))
Patient_2_Prediction=np.zeros((samples,5))
Patient_1_Prediction=np.zeros((samples,5))

#for each finger
for i in range(5):

  #Patient 3
  #fit model to the finger flexion values from training set 

  # now, create model according to optimum alpha value:
  enet_cv_model3 = ElasticNetCV(cv = 10).fit(R_P3_train, Y_train_3_all[:,i])
  enet_tuned3 = ElasticNet(alpha = enet_cv_model3.alpha_).fit(R_P3_train, Y_train_3_all[:,i])
  # additionally, calculate error for (1) training set & (2) test set 
  y_pred_test3 = enet_tuned3.predict(R_P3_l)
  #do smoothing
  smoothed3=smooth(y_pred_test3,15)
  #interpolate the prediction finger angles for this finger/patient
  interpAngle3=zoInterp(smoothed3,winDisp_samples)
  padding3=np.tile(interpAngle3[0],rem_indices)
  totalPrediction3=np.transpose(np.hstack((padding3,interpAngle3)))
  #add to prediction array
  Patient_3_Prediction[:,i]=totalPrediction3

  if i==0:
    filename='p3_f1_model.sav'
    pickle.dump(enet_tuned3, open(filename, 'wb'))
  if i==1:
    filename='p3_f2_model.sav'
    pickle.dump(enet_tuned3, open(filename, 'wb'))
  if i==2:
    filename='p3_f3_model.sav'
    pickle.dump(enet_tuned3, open(filename, 'wb'))
  if i==3:
    filename='p3_f4_model.sav'
    pickle.dump(enet_tuned3, open(filename, 'wb'))
  if i==4:
    filename='p3_f5_model.sav'
    pickle.dump(enet_tuned3, open(filename, 'wb'))

  #Patient 2

  # now, create model according to optimum alpha value:
  enet_cv_model2 = ElasticNetCV(cv = 10).fit(R_P2_train, Y_train_2_all[:,i])
  enet_tuned2 = ElasticNet(alpha = enet_cv_model2.alpha_).fit(R_P2_train, Y_train_2_all[:,i])
  # additionally, calculate error for (1) training set & (2) test set 
  y_pred_test2 = enet_tuned2.predict(R_P2_l)
  #do smoothing
  smoothed2=smooth(y_pred_test2,15)

  #interpolate the prediction finger angles for this finger/patient
  interpAngle2=zoInterp(smoothed2,winDisp_samples)
  padding2=np.tile(interpAngle2[0],rem_indices)
  totalPrediction2=np.transpose(np.hstack((padding2,interpAngle2)))
  #add to prediction array
  Patient_2_Prediction[:,i]=totalPrediction2

  if i==0:
    filename='p2_f1_model.sav'
    pickle.dump(enet_tuned2, open(filename, 'wb'))
  if i==1:
    filename='p2_f2_model.sav'
    pickle.dump(enet_tuned2, open(filename, 'wb'))
  if i==2:
    filename='p2_f3_model.sav'
    pickle.dump(enet_tuned2, open(filename, 'wb'))
  if i==3:
    filename='p2_f4_model.sav'
    pickle.dump(enet_tuned2, open(filename, 'wb'))
  if i==4:
    filename='p2_f5_model.sav'
    pickle.dump(enet_tuned2, open(filename, 'wb'))

  #Patient 1

  # now, create model according to optimum alpha value:
  enet_cv_model1 = ElasticNetCV(cv = 10).fit(R_P1_train, Y_train_1_all[:,i])
  enet_tuned1 = ElasticNet(alpha = enet_cv_model1.alpha_).fit(R_P1_train, Y_train_1_all[:,i])
  # additionally, calculate error for (1) training set & (2) test set 
  y_pred_test1 = enet_tuned1.predict(R_P1_l)
  #do smoothing
  smoothed1=smooth(y_pred_test1,15)

  #interpolate the prediction finger angles for this finger/patient
  interpAngle1=zoInterp(smoothed1,winDisp_samples)
  padding1=np.tile(interpAngle1[0],rem_indices)
  totalPrediction1=np.transpose(np.hstack((padding1,interpAngle1)))
  #add to prediction array
  Patient_1_Prediction[:,i]=totalPrediction1

  if i==0:
    filename='p1_f1_model.sav'
    pickle.dump(enet_tuned1, open(filename, 'wb'))
  if i==1:
    filename='p1_f2_model.sav'
    pickle.dump(enet_tuned1, open(filename, 'wb'))
  if i==2:
    filename='p1_f3_model.sav'
    pickle.dump(enet_tuned1, open(filename, 'wb'))
  if i==3:
    filename='p1_f4_model.sav'
    pickle.dump(enet_tuned1, open(filename, 'wb'))
  if i==4:
    filename='p1_f5_model.sav'
    pickle.dump(enet_tuned1, open(filename, 'wb'))


In [ ]:
Patient_1_Prediction

array([[-0.01035048,  0.01817266, -0.03639564, -0.09625615, -0.06482938],
       [-0.01035048,  0.01817266, -0.03639564, -0.09625615, -0.06482938],
       [-0.01035048,  0.01817266, -0.03639564, -0.09625615, -0.06482938],
       ...,
       [-0.1131203 ,  0.05634154,  0.02131697,  0.15211243,  0.05865611],
       [-0.1131203 ,  0.05634154,  0.02131697,  0.15211243,  0.05865611],
       [-0.1131203 ,  0.05634154,  0.02131697,  0.15211243,  0.05865611]])

In [ ]:
rnnPredictions=np.load("RNNRes.npy")

In [ ]:
p1_rnn=rnnPredictions[0]
p2_rnn=rnnPredictions[1]
p3_rnn=rnnPredictions[2]

p1_f3_rnn=p1_rnn[:,2]
p1_f5_rnn=p1_rnn[:,4]
p2_f3_rnn=p2_rnn[:,2]
p2_f5_rnn=p2_rnn[:,4]

#do smoothing
smoothedp1f3=smooth(p1_f3_rnn,15)
smoothedp1f5=smooth(p1_f5_rnn,15)
smoothedp2f3=smooth(p2_f3_rnn,15)
smoothedp2f5=smooth(p2_f5_rnn,15)

#interpolate the prediction finger angles for this finger/patient
interpAngle1=zoInterp(smoothedp1f3,winDisp_samples)
padding1=np.tile(interpAngle1[0],rem_indices)
totalPrediction1=np.transpose(np.hstack((padding1,interpAngle1)))
#add to prediction array
Patient_1_Prediction[:,2]=totalPrediction1

#interpolate the prediction finger angles for this finger/patient
interpAngle1=zoInterp(smoothedp1f5,winDisp_samples)
padding1=np.tile(interpAngle1[0],rem_indices)
totalPrediction1=np.transpose(np.hstack((padding1,interpAngle1)))
#add to prediction array
Patient_1_Prediction[:,4]=totalPrediction1

#interpolate the prediction finger angles for this finger/patient
interpAngle1=zoInterp(smoothedp2f3,winDisp_samples)
padding1=np.tile(interpAngle1[0],rem_indices)
totalPrediction1=np.transpose(np.hstack((padding1,interpAngle1)))
#add to prediction array
Patient_2_Prediction[:,2]=totalPrediction1

#interpolate the prediction finger angles for this finger/patient
interpAngle1=zoInterp(smoothedp2f5,winDisp_samples)
padding1=np.tile(interpAngle1[0],rem_indices)
totalPrediction1=np.transpose(np.hstack((padding1,interpAngle1)))
#add to prediction array
Patient_2_Prediction[:,4]=totalPrediction1

In [ ]:
Patient_1_Prediction

array([[-0.01035048,  0.01817266,  0.08795108, -0.09625615, -0.04243234],
       [-0.01035048,  0.01817266,  0.08795108, -0.09625615, -0.04243234],
       [-0.01035048,  0.01817266,  0.08795108, -0.09625615, -0.04243234],
       ...,
       [-0.1131203 ,  0.05634154, -0.10196276,  0.15211243,  0.07587013],
       [-0.1131203 ,  0.05634154, -0.10196276,  0.15211243,  0.07587013],
       [-0.1131203 ,  0.05634154, -0.10196276,  0.15211243,  0.07587013]])

In [ ]:
#submit to leaderboard

import numpy as np
from scipy.io import savemat
#create an example submission array

predictions = np.zeros((3,1), dtype=object)
predictions[0,0] = Patient_1_Prediction
predictions[1,0] = Patient_2_Prediction
predictions[2,0] = Patient_3_Prediction

#save the array using the right format
savemat('predictions.mat', {'predicted_dg':predictions})